# Atividade 2 - Visão Computacional - Caio H. Fauza

O entregável de toda esta atividade vai ser um código-fonte em C. 

Encorajamos vocês a fazerem vídeos demonstrando o resultado e a postar (pode ser privadamente) no YouTube



Você deve ter uma folha com o padrão anexo. 
*Dica:* Se não tiver, é possível fazer também com um tablet ou *smartphone*
 
<img src="folha_atividade.png" width=300>

## Parte 1 - calibração

Ouça a explicação do professor sobre o modelo de câmera *pinhole*  e desenhe a medida $f$ que separa o plano focal da pupila da câmera

Detalhe como calculou $f$

Para calcular f, abri o exemplo `draw_circles_video.py`, coloquei a folha paralela a tela em uma distância de 20 cm da câmera, e com as coordenadas dos pontos obtidas pelo código do exemplo, calculei a distância entre as duas circunferências na imagem, por meio da equação da distância entre dois pontos. Após isso, calculei a distância entre as duas circunferências na folha e por meio da seguinte equação, obtive $f$:  
<br>
<center> $\frac{himagem}{f}$ = $\frac{hobjeto}{distância}$</center>
<br>
<center>$\frac{362.00552}{f}$ = $\frac{14}{20}$</center>
<br>
<center>$f$ = 517.150</center>

## Parte 2

Modifique um dos exemplos `draw_circles_video.py` ou `videoplay.py` para passar a ler dados da webcam e identificar o círculo magenta e o círculo ciano, usando o `inRange`

In [26]:
import cv2
#IMPORTANTE: Versão utilizada do OpenCV: 3.4.1

cap = cv2.VideoCapture(0)
import auxiliar as aux

import numpy as np


while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()

    # Our operations on the frame come here
    rgb = np.array(frame, dtype=np.uint8)
    hsv = cv2.cvtColor(rgb, cv2.COLOR_BGR2HSV)
    
    #cyan
    #004a9f
    hsvc1, hsvc2 = aux.ranges("#004a9f")
    
    #magenta
    #841a2b
    hsvm1, hsvm2 = aux.ranges("#841a2b")
    
    cyanmask = cv2.inRange(hsv, hsvc1, hsvc2) 
    magentamask = cv2.inRange(hsv, hsvm1, hsvm2)
    
    cyancolor = cv2.morphologyEx(cyanmask,cv2.MORPH_CLOSE,np.ones((10, 10)))
    selectcyan = cv2.bitwise_and(rgb, rgb, mask=cyancolor)
    
    magentacolor = cv2.morphologyEx(magentamask,cv2.MORPH_CLOSE,np.ones((10, 10)))
    combinada = cv2.bitwise_or(cyancolor, magentacolor)
    selectfinal = cv2.bitwise_and(rgb, rgb, mask=combinada)
    

    cv2.imshow('frame', selectfinal)    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
    

cap.release()
cv2.destroyAllWindows()

## Parte 3

Assumindo que a folha se mantém sempre paralela ao plano de imagem da câmera, imprima a distância entre a folha e sua câmera

In [11]:
cap = cv2.VideoCapture(0)

def distancia(x, x0, y, y0):
    pixel = (((x-x0)**2) + ((y-y0)**2))**(1/2)
    hobjeto = 14
    f = 517.150
    return f*(hobjeto/pixel)

def auto_canny(image, sigma=0.33):
    
    v = np.median(image)


    lower = int(max(0, (1.0 - sigma) * v))
    upper = int(min(255, (1.0 + sigma) * v))
    edged = cv2.Canny(image, lower, upper)

    return edged

while(True):
    ret, frame = cap.read()
    
    
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray,(5,5),0)
    bordas = auto_canny(blur)
    
    circles = []
    circles=cv2.HoughCircles(bordas,cv2.HOUGH_GRADIENT,2,40,param1=50,param2=100,minRadius=5,maxRadius=60)
    
    bordas_color = cv2.cvtColor(bordas, cv2.COLOR_GRAY2BGR)
    font = cv2.FONT_HERSHEY_SIMPLEX
    
    
    if circles is not None:
        circles = np.uint16(np.around(circles))
        for i in circles[0,:]:
            cv2.circle(bordas_color,(i[0],i[1]),i[2],(0,255,0),2)
            cv2.circle(bordas_color,(i[0],i[1]),2,(0,0,255),3)
        
        if len(circles[0]) == 2:
            x, y, r1 = circles[0][0]
            x0, y0, r0 = circles[0][1]
            cv2.putText(bordas_color, str(distancia(int(x), int(x0), int(y), int(y0))), (0,50), font, 2, (20, 100, 50), 2, cv2.LINE_AA)
        
    cv2.imshow('frame', bordas_color)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
        
   


cap.release()
cv2.destroyAllWindows()

## Parte 4

Trace uma linha entre os centros do círculo magenta e do círculo ciano.

Imprima na tela o ângulo entre esta linha e a horizontal

In [13]:
import math

cap = cv2.VideoCapture(0)


def auto_canny(image, sigma=0.33):
    
    v = np.median(image)


    lower = int(max(0, (1.0 - sigma) * v))
    upper = int(min(255, (1.0 + sigma) * v))
    edged = cv2.Canny(image, lower, upper)

    return edged

while(True):
    ret, frame = cap.read()
    
    
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray,(5,5),0)
    bordas = auto_canny(blur)
    
    circles = []
    circles=cv2.HoughCircles(bordas,cv2.HOUGH_GRADIENT,2,40,param1=50,param2=100,minRadius=5,maxRadius=60)
    
    bordas_color = cv2.cvtColor(bordas, cv2.COLOR_GRAY2BGR)
    font = cv2.FONT_HERSHEY_SIMPLEX
    
    
    
    
    
    if circles is not None:
        circles = np.uint16(np.around(circles))
        for i in circles[0,:]:
            cv2.circle(bordas_color,(i[0],i[1]),i[2],(0,255,0),2)
            cv2.circle(bordas_color,(i[0],i[1]),2,(0,0,255),3)
            
        
        if len(circles[0]) == 2:
            x, y, r1 = circles[0][0]
            x0, y0, r0 = circles[0][1]
        
            linecircle = cv2.line(bordas_color,(x0,y0),(x, y),(255,0,0),5)
            cv2.putText(bordas_color, str((180/math.pi)*(math.atan2(int(y)-int(y0), int(x)-int(x0)))), (0,50), font, 2, (20, 100, 50), 2, cv2.LINE_AA)
    
    cv2.imshow('frame', bordas_color)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
   


cap.release()
cv2.destroyAllWindows()

## Parte 5

Usando transformada de Hough, desenhe um círculo sobre o círculo ciano e outro sobre o círculo magenta.

**Desafio bônus**: ser capaz de eliminar circulos espúrios (aqueles que não são os da folha)

In [21]:
cap = cv2.VideoCapture(0)

def auto_canny(image, sigma=0.33):
    
    v = np.median(image)


    lower = int(max(0, (1.0 - sigma) * v))
    upper = int(min(255, (1.0 + sigma) * v))
    edged = cv2.Canny(image, lower, upper)

    return edged

while(True):
    ret, frame = cap.read()
    
    
    rgb = np.array(frame, dtype=np.uint8)
    hsv = cv2.cvtColor(rgb, cv2.COLOR_BGR2HSV)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray,(5,5),0)
    bordas = auto_canny(blur)
    
    
    #cyan
    #004a9f
    hsvc1, hsvc2 = aux.ranges("#004a9f")
    
    #magenta
    #841a2b
    hsvm1, hsvm2 = aux.ranges("#841a2b")
    
    cyanmask = cv2.inRange(hsv, hsvc1, hsvc2) 
    magentamask = cv2.inRange(hsv, hsvm1, hsvm2)
    
    cyancolor = cv2.morphologyEx(cyanmask,cv2.MORPH_CLOSE,np.ones((10, 10)))
    
    
    magentacolor = cv2.morphologyEx(magentamask,cv2.MORPH_CLOSE,np.ones((10, 10)))
    combinada = cv2.bitwise_or(cyancolor, magentacolor)
    selectfinal = cv2.bitwise_and(rgb, rgb, mask=combinada)
    
    
    circles = []
    circles=cv2.HoughCircles(bordas,cv2.HOUGH_GRADIENT,2,40,param1=50,param2=100,minRadius=5,maxRadius=60)
    
    
    bordas_color = cv2.cvtColor(bordas, cv2.COLOR_GRAY2BGR)
    
    
    if circles is not None:
        circles = np.uint16(np.around(circles))
        for i in circles[0,:]:
                cv2.circle(selectfinal,(i[0],i[1]),i[2],(0,255,0),2)
                cv2.circle(selectfinal,(i[0],i[1]),2,(0,0,255),3)
        
        
        
    cv2.imshow('frame', selectfinal)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
   


cap.release()
cv2.destroyAllWindows()

## Parte 6

Usando `SIFT`, identifique o escrito *Insper* na folha

In [25]:
cap = cv2.VideoCapture(0)

while(True):
    ret, frame = cap.read()
    
    MIN_MATCH_COUNT = 10
    img_original = cv2.imread('logo1.png',0) #Utilizar logo.png ou logo1.png
    img_cena = frame
    
    
    original_rgb = cv2.cvtColor(img_original, cv2.COLOR_GRAY2RGB)
    out = img_cena.copy()
    
    sift = cv2.xfeatures2d.SIFT_create()

    
    kp1, des1 = sift.detectAndCompute(img_original ,None)
    kp2, des2 = sift.detectAndCompute(img_cena,None)
    
    FLANN_INDEX_KDTREE = 0
    index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
    search_params = dict(checks = 50)
    
    
    flann = cv2.FlannBasedMatcher(index_params, search_params)
    matches = flann.knnMatch(des1,des2,k=2)
    

    good = []
    for m,n in matches:
        if m.distance < 0.7*n.distance:
            good.append(m)
            
    if len(good)>MIN_MATCH_COUNT:
        src_pts = np.float32([ kp1[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
        dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good ]).reshape(-1,1,2)
        
        M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,5.0)
        matchesMask2 = mask.ravel().tolist()
    
        h,w = img_original.shape
        pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)
        dst = cv2.perspectiveTransform(pts,M)
    
        img2b = cv2.polylines(out,[np.int32(dst)],True,(255,0,0),3, cv2.LINE_AA)
    
    
    
    else:
        pass

    draw_params = dict(matchColor = (0,255,0), singlePointColor = None, flags = 2)
    
    


    cv2.imshow('frame', out)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()   

<br> 
<center><h1>Método alternativo para parte 5</h1></center>
<br>

In [20]:
cap = cv2.VideoCapture(0)

while(True):
    ret, frame = cap.read()

    magentamaior = np.array([150, 50, 50])
    magentamenor = np.array([150, 255, 255])
    
    cyanmaior = np.array([106, 50, 50])
    cyanmenor= np.array([106, 50, 50])
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    hsvm1, hsvm2 = aux.ranges("#841a2b")
    hsvc1, hsvc2 = aux.ranges("#004a9f")
    
    
    maskmagenta = cv2.inRange(hsv, hsvm1, hsvm2)
    maskcyan = cv2.inRange(hsv, hsvc1, hsvc2)
    
    segmentadomagenta = cv2.morphologyEx(maskmagenta,cv2.MORPH_CLOSE,np.ones((4, 4)))
    contornosmagenta, arvore, h = cv2.findContours(segmentadomagenta, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE, )
    
    segmentadocyan = cv2.morphologyEx(maskcyan,cv2.MORPH_CLOSE,np.ones((4, 4)))
    contornoscyan, arvore2, h2 = cv2.findContours(segmentadocyan, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE) 
   

    contornosm_img = cv2.drawContours(frame.copy(), arvore, -1, [0, 0, 255], 3);
    contornosc_img = cv2.drawContours(frame.copy(), arvore2, -1, [0, 0, 255], 3);
    
    juntos = cv2.bitwise_or(contornosm_img, contornosc_img)
    cv2.imshow('frame', juntos)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()   